# BentoML Quick Start

- https://docs.bentoml.org/en/latest/tutorial.html




- 설치 방법

- 2022-08-29 기준으로, fastapi 최신 버전에서 문제 발생하고 있음
- 임시로 fastapi==0.75 를 사용할 것 (https://discuss.ray.io/t/importerror-cannot-import-name-paramspec-from-typing-extensions-when-creating-a-cluster-on-azure/6869)

```bash
pip3 install bentoml scikit-learn pandas
conda install fastapi==0.75

```

## Quick test

In [7]:
from fastapi import routing

In [11]:
import bentoml

from sklearn import svm
from sklearn import datasets

# Load training data set
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Train the model
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

# Save model to the BentoML local model store
saved_model = bentoml.sklearn.save_model("iris_clf", clf)
print(f"Model saved: {saved_model}")

# Model saved: Model(tag="iris_clf:zy3dfgxzqkjrlgxi")

Model saved: Model(tag="iris_clf:lsdv33rhucynuzrg")


In [1]:
! bentoml models get iris_clf:latest

name: iris_clf                                                                  
version: lsdv33rhucynuzrg                                                       
module: bentoml.sklearn                                                         
labels: {}                                                                      
options: {}                                                                     
metadata: {}                                                                    
context:                                                                        
  framework_name: sklearn                                                       
  framework_versions:                                                           
    scikit-learn: 1.1.2                                                         
  bentoml_version: 1.0.4                                                        
  python_version: 3.9.13                                                        
signatures:                 

In [2]:
! bentoml models list


 Tag                             Module           Size      Creation Time       
 pytorch_mnist:2nd6rfrhus6fazrg  bentoml.pytorch  1.30 MiB  2022-08-29 23:13:59 
 iris_clf:lsdv33rhucynuzrg       bentoml.sklearn  5.79 KiB  2022-08-29 22:42:01 
 iris_clf:lmrxdybhucynuzrg       bentoml.sklearn  5.79 KiB  2022-08-29 22:41:59 
 iris_clf:e2nvzrrhucynuzrg       bentoml.sklearn  5.79 KiB  2022-08-29 22:40:31 


In [3]:
! bentoml models --help

Usage: bentoml models [OPTIONS] COMMAND [ARGS]...

  Model Subcommands Groups

Options:
  -h, --help  Show this message and exit.

Commands:
  delete  Delete Model in local model store.
  export  Export a Model to an external archive file
  get     Print Model details by providing the model_tag
  import  Import a previously exported Model archive file
  list    List Models in local store
  pull    Pull Model from a yatai server.
  push    Push Model to a yatai server.


### Prediction

In [4]:
import bentoml

loaded_model = bentoml.sklearn.load_model("iris_clf:latest")

loaded_model.predict([[5.9, 3. , 5.1, 1.8]])  # => array(2)

array([2])

In [5]:
model = bentoml.sklearn.load_model("iris_clf:lsdv33rhucynuzrg")

# Alternatively, use `latest` to find the newest version
model = bentoml.sklearn.load_model("iris_clf:latest")

In [6]:
model

SVC()

## Serving the model

### service.py

In [15]:
# service.py

import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    result = iris_clf_runner.predict.run(input_series)
    return result

In [9]:
! bentoml serve service.py:svc --reload 

2022-08-30T10:55:26+0900 [INFO] [cli] Starting development BentoServer from "service.py:svc" running on http://0.0.0.0:3000 (Press CTRL+C to quit)
2022-08-30 10:55:26 circus[34548] [INFO] Loading the plugin...
2022-08-30 10:55:26 circus[34548] [INFO] Endpoint: 'tcp://127.0.0.1:61384'
2022-08-30 10:55:26 circus[34548] [INFO] Pub/sub: 'tcp://127.0.0.1:61385'
2022-08-30T10:55:26+0900 [INFO] [observer] Watching directories: ['/Users/jpark/WorkDevEdgeAI/cloud-edge-aicontainers/v2/approaches/bentoML/00_intro', '/Users/jpark/bentoml/models']
^C


In [10]:
curl -X POST -H "content-type: application/json" --data "[[5.9, 3, 5.1, 1.8]]" http://127.0.0.1:3000/classify

SyntaxError: invalid syntax (4128549827.py, line 1)

### [python] Send prediction request with an HTTP client

In [16]:
import requests
requests.post(
    "http://127.0.0.1:3000/classify",
    headers={"content-type": "application/json"},
    data="[[5.9, 3, 5.1, 1.8]]").text

'[2]'

### [curl] Send prediction request with an HTTP client

```bash
curl \
  -X POST \
  -H "content-type: application/json" \
  --data "[[5.9, 3, 5.1, 1.8]]" \
  http://127.0.0.1:3000/classify
```




In [19]:
import bentoml

iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()
iris_clf_runner.init_local()
iris_clf_runner.predict.run([[5.9, 3., 5.1, 1.8]])

'Runner.init_local' is for debugging and testing only


array([2])